<a href="https://colab.research.google.com/github/armansheikhibiglari/PRACTICE-2/blob/main/NLPSUMMERIZE_exam_assinment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.stem import PorterStemmer as stemmer
from nltk import FreqDist
from nltk.classify import apply_features
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
import re
from nltk.cluster.util import cosine_distance
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
import nltk
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [3]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import numpy as np

nltk.download("punkt")
nltk.download("stopwords")

class HierarchicalSummarizer:
    def __init__(self, context_window=4000):
        self.context_window = context_window
        self.stop_words = set(stopwords.words("english"))

    def normalize_sentence(self, sentence):
        """Tokenize and remove stop words."""
        return [word.lower() for word in word_tokenize(sentence) if word.isalnum() and word.lower() not in self.stop_words]

    def cosine_similarity(self, vec1, vec2):
        """Calculate cosine similarity between two vectors."""
        dot_product = np.dot(vec1, vec2)
        norm_vec1 = np.linalg.norm(vec1)
        norm_vec2 = np.linalg.norm(vec2)
        return dot_product / (norm_vec1 * norm_vec2) if norm_vec1 and norm_vec2 else 0

    def sentence_vector(self, sentence, vocab):
        """Create a vector representation of a sentence based on a vocabulary."""
        vector = np.zeros(len(vocab))
        for word in sentence:
            if word in vocab:
                vector[vocab.index(word)] += 1
        return vector

    def summarize(self, text, target_length):
        """Summarize the text to the target length."""
        sentences = sent_tokenize(text)
        normalized_sentences = [self.normalize_sentence(sentence) for sentence in sentences]
        vocab = list(set(word for sentence in normalized_sentences for word in sentence))
        sentence_vectors = [self.sentence_vector(sentence, vocab) for sentence in normalized_sentences]

        # Rank sentences by importance using cosine similarity
        scores = [sum(self.cosine_similarity(vec, sentence_vectors[j]) for j in range(len(sentences)) if i != j)
                  for i, vec in enumerate(sentence_vectors)]
        ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)

        # Return the top sentences based on the target length
        return ' '.join(sentence for _, sentence in ranked_sentences[:target_length])

    def hierarchical_summarize(self, text):
        """Perform hierarchical summarization for large texts."""
        tokens = word_tokenize(text)
        print(f"Original document length (tokens): {len(tokens)}")

        # Check if the document is already within the context window
        if len(tokens) <= self.context_window:
            return self.summarize(text, target_length=int(len(tokens) * 0.1))  # Proportional summary for small texts

        # Slice and summarize in chunks
        slices = [tokens[i:i + self.context_window] for i in range(0, len(tokens), self.context_window)]
        summarized_slices = [self.summarize(' '.join(slice), target_length=int(len(slice) * 0.1)) for slice in slices]

        # Combine summaries and shrink if necessary
        combined_summary = ' '.join(summarized_slices)
        if len(word_tokenize(combined_summary)) > self.context_window:
            return self.hierarchical_summarize(combined_summary)

        return combined_summary

    def save_summary(self, summary, filename="summary.txt"):
        """Save the summary to a file."""
        with open(filename, 'w') as f:
            f.write(summary)

    def generate_query(self, summary):
        """Generate a query based on the summary."""
        # This can be refined depending on the query's desired form.
        query = "Query: " + summary[:50] + "..."  # A simple example query based on the summary
        return query


# Example usage
documents = [
    "The validation set is used during the training process to assess the performance of a machine learning model. "
    "The purpose of the validation set is to provide an unbiased evaluation of the model's generalization ability. "
    "It helps to tune the model parameters and prevent overfitting. In this context, it is a crucial element of model evaluation."
]

summarizer = HierarchicalSummarizer()

for i, doc in enumerate(documents, start=1):
    print(f"\nDocument {i} Summary:")
    summary = summarizer.hierarchical_summarize(doc)
    print(summary)

    # Save the summary to a file
    summarizer.save_summary(summary, filename=f"document_{i}_summary.txt")

    # Generate a query based on the summary
    query = summarizer.generate_query(summary)
    print(f"Generated Query: {query}")



Document 1 Summary:
Original document length (tokens): 62
The purpose of the validation set is to provide an unbiased evaluation of the model's generalization ability. In this context, it is a crucial element of model evaluation. The validation set is used during the training process to assess the performance of a machine learning model. It helps to tune the model parameters and prevent overfitting.
Generated Query: Query: The purpose of the validation set is to provide an...


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
